In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import remove_too_long,type_score
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache


In [2]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [3]:
df_ipcc['ipcc']=df_ipcc.apply(lambda row: {'wg': row['wg'].replace('wg',''), 'chapter': int(row['chap'][-2:].replace('_',''))}, axis=1)

In [4]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'ipcc': lambda x: list(x), 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [5]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [6]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [7]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: {'chapter': [x.replace('chap_','') for x in row['chap']]}, axis=1)

In [8]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'ipbes': lambda x: list(x), 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipbes_group=df_ipbes_group.reset_index()

In [9]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [10]:
df_ipbes_group

,doi,ipbes,countries,freq
0,10.1001/archpedi.161.7.718,[{'chapter': ['5']}],None,1
1,10.1002/(sici)1099-0771(199612)9:4<231::aid-bd...,[{'chapter': ['4']}],[US],1
2,10.1002/16-0188.1,"[{'chapter': ['2.1', '5']}, {'chapter': ['2.1'...",[US],2
3,10.1002/2013ef000188,[{'chapter': ['2.1']}],[FR],1
4,10.1002/2013gb004654,[{'chapter': ['2.2']}],[DE],1
...,...,...,...,...
6575,10.7930/j0x63jt0,[{'chapter': ['glossary']}],None,1
6576,10.9755/ejfa.v25i1.15376,[{'chapter': ['3']}],[US],1
6577,10.9758/cpn.2015.13.3.239,[{'chapter': ['2.3']}],[TR],1
6578,10.9774/gleaf.978-1-907643-06-4_5,[{'chapter': ['2.1']}],None,1


In [11]:
df_ipcc_group

,doi,ipcc,countries,freq
0,10.1001/jama.2012.108940,"[{'wg': '2', 'chapter': 6}]",[US],1
1,10.1001/jama.2014.13186,"[{'wg': '2', 'chapter': 7}]","[US, GB]",1
2,10.1001/jama.2014.15715,"[{'wg': '2', 'chapter': 7}]",[US],1
3,10.1001/jama.2020.1097,"[{'wg': '2', 'chapter': 7}]",[US],1
4,10.1001/jama.2020.15398,"[{'wg': '2', 'chapter': 4}, {'wg': '2', 'chapt...",[US],3
...,...,...,...,...
53107,10.9753/icce.v36.risk.59,"[{'wg': '2', 'chapter': 16}]","[US, GB, JP]",1
53108,10.9770/jesi.2017.4.4(13),"[{'wg': '2', 'chapter': 18}]",None,1
53109,10.9774/gleaf.3062.2001.wi.00007,"[{'wg': '2', 'chapter': 4}]",None,1
53110,10.9774/gleaf.4700.2015.ju.00009,"[{'wg': '2', 'chapter': 18}]",None,1


In [12]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes

,doi,ipcc,countries_x,freq_x,ipbes,countries_y,freq_y
0,10.1001/jama.2012.108940,"[{'wg': '2', 'chapter': 6}]",[US],1.0,NaN,NaN,NaN
1,10.1001/jama.2014.13186,"[{'wg': '2', 'chapter': 7}]","[US, GB]",1.0,NaN,NaN,NaN
2,10.1001/jama.2014.15715,"[{'wg': '2', 'chapter': 7}]",[US],1.0,NaN,NaN,NaN
3,10.1001/jama.2020.1097,"[{'wg': '2', 'chapter': 7}]",[US],1.0,NaN,NaN,NaN
4,10.1001/jama.2020.15398,"[{'wg': '2', 'chapter': 4}, {'wg': '2', 'chapt...",[US],3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
58515,10.7930/j0x63jt0,NaN,NaN,NaN,[{'chapter': ['glossary']}],None,1.0
58516,10.9755/ejfa.v25i1.15376,NaN,NaN,NaN,[{'chapter': ['3']}],[US],1.0
58517,10.9758/cpn.2015.13.3.239,NaN,NaN,NaN,[{'chapter': ['2.3']}],[TR],1.0
58518,10.9774/gleaf.978-1-907643-06-4_5,NaN,NaN,NaN,[{'chapter': ['2.1']}],None,1.0


In [13]:
update_bso_publications()

2024-02-21 17:18:55,120 | code_utils.BSO | DEBUG | update bso publications files


DEBUG:code_utils.BSO:update bso publications files
c:\Users\haallat\Documents\teds\code_utils\BSO.py:34: DtypeWarning: Columns (17,18,21,22,23,24,25,27,28,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bso = pd.read_csv('https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump/bso-publications-latest.csv.gz', sep=';')


2024-02-21 17:21:37,360 | code_utils.BSO | DEBUG | writing 1810971 dois info from bso publications


DEBUG:code_utils.BSO:writing 1810971 dois info from bso publications


In [14]:
bso=pd.DataFrame(get_bso_publications())

In [15]:
df_bso=bso.transpose().reset_index()

In [16]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [17]:
df_bso

,doi,rors,bso_local_affiliations
0,10.1109/ntms.2018.8328751,[00e96v939],[199119751]
1,10.1504/ijaac.2018.10010855,[00e96v939],[199119751]
2,10.2139/ssrn.3544866,[00e96v939],[199119751]
3,10.1201/9781003026686,[00e96v939],[199119751]
4,10.23919/acc.2018.8431168,[00e96v939],[199119751]
...,...,...,...
1810966,10.1101/2021.02.05.429917,[],[]
1810967,10.1101/2021.04.23.441180,"[003vg9w96, 03xjwb503]","[180070039, 130002850FULL, sde, gs-biosphera, ..."
1810968,10.1101/2020.04.15.044065,[],[]
1810969,10.1364/optica.397172,[02en5vm52],"[180089013, 130023385]"


In [18]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left')

In [19]:
def get_countries(row):
    if (isinstance(row.countries_x,list)==False)&(isinstance(row.countries_y,list)==False):
        return None
    elif (isinstance(row.countries_x,list))&(isinstance(row.countries_y,list)==False):
        return row.countries_x
    elif (isinstance(row.countries_x,list)==False)&(isinstance(row.countries_y,list)):
        return row.countries_y
    else:
        return row.countries_x

In [20]:
df_ipcc_ipbes_bso['countries']=df_ipcc_ipbes_bso.apply(lambda row: get_countries(row),axis=1)

In [21]:
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso[['doi', 'ipcc', 'ipbes', 'countries', 'rors', 'bso_local_affiliations']]

In [22]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

53358 data in cached openalex data
6608 data in cached openalex data


In [23]:
df_cache

,doi,OpenAlex_data
0,10.1098/rstl.1686.0026,"[{'id': 'https://openalex.org/W4244279015', 'd..."
1,10.1098/rstl.1735.0014,"[{'id': 'https://openalex.org/W4252425265', 'd..."
2,10.1098/rstl.1861.0001,"[{'id': 'https://openalex.org/W3091998362', 'd..."
3,10.1017/s0080456800017592,"[{'id': 'https://openalex.org/W2314134964', 'd..."
4,10.2307/2992177,"[{'id': 'https://openalex.org/W256209766', 'do..."
...,...,...
58788,10.9774/gleaf.978-1-907643-06-4_5,"[{'id': 'https://openalex.org/W3124428460', 'd..."
58789,10.9783/9780812295948-002,"[{'id': 'https://openalex.org/W3137011924', 'd..."
58790,10.2172/2283221,"[{'id': 'https://openalex.org/W4391458189', 'd..."
58791,10.1016/b978-0-12-812362-1.00002-3,"[{'id': 'https://openalex.org/W3107290679', 'd..."


In [24]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [25]:
dict_doi_bso=[]

In [26]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [ ]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [ ]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', lines=True)

In [ ]:
current_dict = dd.to_dict(orient='records')

In [ ]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [ ]:
df_elastic_input=pd.DataFrame(current_dict)

In [ ]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [ ]:
del df_elastic_input['openalex_abstract_inverted_index']

In [ ]:
len(df_elastic_input)

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [ ]:
df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'